# **Аналитика текстового контента (JSON) карточек товаров интернет магазина**
ТЗ: https://docs.google.com/document/d/1KPTUZRcN1clU2ZgIpq-AM5baJ6Q8FhZP9AcjQLLU4c4/edit#heading=h.jy00ifncyqyc

**Импорт необходимых модулей**



In [212]:
!pip install yargy ipymarkup

from google.colab import files
import json
import os
import re
from random import randint, seed, sample
from IPython.display import display

from ipymarkup import show_span_ascii_markup as show_markup

from yargy import (
    Parser,
    or_, rule
)
from yargy.pipelines import morph_pipeline
from yargy.predicates import (
    eq, in_, dictionary,
    type, gram
)
from yargy.tokenizer import MorphTokenizer
from yargy import interpretation as interp
from yargy.interpretation import fact, attribute



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Открытие исходного json файла с товарами**

In [213]:
def open_json_file(filepath: str) -> json:
    print(f'Файл с данными для загрузки: "{filepath}"')
    try:
        with open(filepath, 'r', encoding='utf8') as f:
            data = json.load(f)
            print(f'файл json загружен ({len(data)} строк)')
            return data
    except:
        print(f'не удалось загрузить json файл')

json_file = '/content/import_json/wildberries items middle.json'
catalog = open_json_file(json_file)

Файл с данными для загрузки: "/content/import_json/wildberries items middle.json"
файл json загружен (1232 строк)


**Создание базовых каталогов на основе исходного файла с товарами (опционально)**

In [214]:
def save_catalog(name:str='unnamed', data:set=()) -> bool:
  try:
    data = sorted(list(data))
    pathfile = f'catalogs/{name}.txt'
    with open(pathfile, 'w') as f:
      for row in data:
        if row == '':
          continue
        f.write(f'{row.strip().lower()}\n')
    print(f'Каталог "{name}" ({len(data)} записей) - сохранен!')
  except:
    print(f'ОШИБКА: Каталог "{name}" ({len(data)} записей) - не сохранен!')

def create_catalog_colors(catalog:json=[]) -> set:
  catalog_colors = set()
  for product in catalog:
    colors = product['colors']
    for color in colors:
      if ',' in color:
        for c in color.split(', '):
          c = re.sub('[^a-zA-ZА-Яа-я-]', '', c).strip()
          if c not in catalog_colors and len(c) > 3:
            catalog_colors.add(c)
      else:
        color = re.sub('[^a-z-]', '', color).strip()
        if color not in catalog_colors and len(color) > 3:
          catalog_colors.add(color)
  return catalog_colors

def create_catalog_materials(catalog:json=[]) -> set:
  catalog_materials = set()
  for product in catalog:
    materials = product['consists'].split(', ')
    for material in materials:
      if material not in catalog_materials:
        catalog_materials.add(material)
  return catalog_materials

def create_catalog_items(catalog:json=[]) -> set:
  catalog_items = set()
  for product in catalog:
    item = product['subjectName']
    if item not in catalog_items:
      catalog_items.add(item)
  return catalog_items

catalog_colors = create_catalog_colors(catalog=catalog)
catalog_items = create_catalog_items(catalog=catalog)
catalog_materials = create_catalog_materials(catalog=catalog)

save_catalog('color', catalog_colors)
save_catalog('items', catalog_items)
save_catalog('materials', catalog_materials)

Каталог "color" (84 записей) - сохранен!
Каталог "items" (344 записей) - сохранен!
Каталог "materials" (1757 записей) - сохранен!


**Загрузка базовых каталогов**

In [215]:
def load_lines(path):
    with open(path) as file:
        for line in file:
            yield line.rstrip('\n')

COLORS = set(load_lines('catalogs/color.txt'))
GENDER = set(load_lines('catalogs/gender.txt'))
ITEMS = set(load_lines('catalogs/items.txt'))
MATERIALS = set(load_lines('catalogs/materials.txt'))
SEASONS = set(load_lines('catalogs/seasons.txt'))

# AREAS = set(load_lines('dicts/areas.txt'))
seed(10)
print('Случайные значения:')
print('> цвета:', sample(sorted(COLORS), 10))
print('> гендеры:', sample(sorted(GENDER), 2))
print('> предметы:', sample(sorted(ITEMS), 10))
print('> материалы:', sample(sorted(MATERIALS), 10))
print('> сезоны:', sample(sorted(SEASONS), 5))


Случайные значения:
> цвета: ['хаки', 'антрацитовый', 'светло-серыймеланж', 'сине-серый', 'ярко-зеленый', 'cинийлн', 'камуфляж', 'серый', 'синий', 'мокко']
> гендеры: ['женщина', 'мужчина']
> предметы: ['слитные купальники', 'рахат-лукум', 'мясо сушеное', 'воздушные шарики', 'крючки для штор', 'ножи кухонные', 'бритвы электрические', 'печенье спортивное', 'защита стволов деревьев', 'туалетная бумага']
> материалы: ['кунжутное масло 1%', 'лофант', 'метилхлороизотиазолинон', 'картофель 4%', 'экстракт ягод бузины чёрной', 'соль морская', 'калия цетилфосфат', 'мыльная основа глицериновая без sls 97%', 'гидроксипропилгуар', 'спандекс 13%']
> сезоны: ['лето', 'осень-зима', 'весна', 'зима', 'осень']


**Просмотр карточки случайного товара** (по его номеру по порядку в каталоге )

In [217]:
products_count = len(catalog)
product_num = randint(1, products_count)
product_id = product_num-1

def print_source_data(product_num) -> bool:
  
  def get_chars_str(chars:list) -> str:
    chars_str = ''
    num = 0
    for c in chars:
        num += 1
        k = c['k']
        v = c['v']
        chars_str += f'  > {num}) "{k}": [{v}]\n'
    if chars_str != '':
        chars_str = chars_str[:-1]
    return chars_str
  
  def get_colors_str(colors:list) -> str:
        colors_str = ", ".join(colors)
        if colors_str == '':
            colors_str = '-нет-'
        return colors_str

  name = catalog[product_id]['name']
  description = catalog[product_id]['description']
  subjectName = catalog[product_id]['subjectName']
  chars = catalog[product_id]['chars']
  chars_str = get_chars_str(chars)
  colors = catalog[product_id]['colors']
  colors_str = get_colors_str(colors=colors)

  print(f'##### КАРТОЧКА ТОВАРА #####')
  print(f'> Название товара: [{name}]')
  print(f'> Описание товара:\n[{description}]')
  print(f'> Название предмета товара: [{subjectName}]')
  print(f'> Характеристики товара:\n{chars_str}')
  print(f'> Цвета: [{colors_str}]')
  print(f'----- товар {product_num}/{products_count} -----')

print_source_data(product_num)


##### КАРТОЧКА ТОВАРА #####
> Название товара: [Гигиенические тампоны Compak Super Plus 16шт.]
> Описание товара:
[Чистота останется с тобой, как привязанная! У тампонов Tampax есть гладкий аппликатор, который помогает гигиенично ввести тампон, не касаясь его руками. Одновременно аппликатор позволяет правильно расположить тампон там, где ты практически не будешь его ощущать. Tampax Compak - самый маленький тампон с аппликатором, который так легко спрятать в руке.]
> Название предмета товара: [Тампоны гигиенические]
> Характеристики товара:
  > 1) "Количество предметов в упаковке": [16 шт.]
  > 2) "Транспортировка": [не опасный груз]
  > 3) "Вес товара с упаковкой (г)": [89 г]
  > 4) "Возрастные ограничения": [18+]
  > 5) "Высота упаковки": [6.9 см]
  > 6) "Глубина упаковки": [6.9 см]
  > 7) "Упаковка": [картонная коробка]
  > 8) "Форма упаковки": [без давления]
  > 9) "Ширина упаковки": [9 см]
  > 10) "Страна производства": [Венгрия]
  > 11) "Комплектация": [тампоны]
> Цвета: [темно-си

**Описание моделей yargy**

In [218]:
# TOKENIZER = MorphTokenizer()
# list(TOKENIZER('Крем-спрей'))

INT = type('INT')
NOUN = gram('NOUN')
ADJF = gram('ADJF')
PRTF = gram('PRTF')
GENT = gram('gent')
DOT = eq('.')


def show_matches(rule, log=False, text:str=''):
    facts = []
    parser = Parser(rule)
    # for line in lines:
    matches = parser.findall(text)
    matches = sorted(matches, key=lambda _: _.span)
    spans = [_.span for _ in matches]
    
    for m in matches:
      facts.append(m.fact)
    if log:
      show_markup(text, spans)
      # print(f' -> найдено {len(facts)}: [{facts}]')
    return facts

        # if matches:
        #     facts = [_.fact for _ in matches]
        #     if len(facts) == 1:
        #         facts = facts[0]
        #     display(facts)
        # print(matches)

**Правила для yargy (rules)**

In [222]:
def normalize_float(value):
    value = re.sub('[\s,.]+', '.', value)
    return float(value)

FLOAT = rule(
    INT,
    in_('.,'),
    INT
).interpretation(
    interp.custom(normalize_float)
)

DIGIT = INT.interpretation(
    interp.custom(int)
)

ADJ = ADJF.interpretation(
    interp.normalized()
)


COLOR = pipeline(COLORS)
COLOR_RULE = rule(COLOR).interpretation(
    interp.normalized()
)

show_matches(FLOAT, log=True, text='~ 3,14')
show_matches(DIGIT, log=True, text='15')
show_matches(ADJ, log=True, text='Красивый')
show_matches(COLOR_RULE, log=True, text='желтый')



~ 3,14
  ────
15
──
Красивый
────────
желтый
──────


['жёлтый']

**Тестирование поиска**

In [223]:
texts = []
for product in catalog:
  title = product['name']
  texts.append(title)

seed(50)
for text in sample(texts, 10):
    show_matches(DIGIT, log=True, text= text)
    # show_matches(COLOR_RULE, log=True, text=text.lower())


Интимный ликбез с родителями и без. Книга о том откуда берутся дети. 
Энциклопедия для детей.
Средство VISIBLE REPAIR для поврежденных волос с пантенолом, 750 мл
                                                             ───   
Ночной антивозрастной крем для лица "Возраст Эксперт 65+", против 
                                                     ──           
морщин, питательный, 50 мл
                     ──   
Носки мужские антибактериальные ароматизированные набор в подарочной 
упаковке.
Протеиновый батончик без сахара "Кокос", 50гр х 12 шт / Cпортивные 
                                         ──     ──                 
батончики FitnesShock
Носки мужские набор / в подарок / с принтом / с рисунком / в 
подарочной коробке, 5 пар /хлопок
                    ─            
FLORMAR / VIOLET Набор Водостойких Матовых карандашей для Губ с 
Точилкой 12 штук Нюдовые оттенки
         ──                     
Влажная туалетная бумага с клапаном "SENSO BABY", Можно смывать, Алоэ 
и Ромашка, 6

**Анализ данных карточки случайного товара**

In [224]:
products_count = len(catalog)
product_num = randint(1, products_count)

def get_colors(words:str) -> bool:
  print(f'> Цвета:')
  adj = show_matches(COLOR_RULE, text=words.lower())
  print(f'  > количество: [{len(adj)}]')
  print(f'  > слова: [{adj}]')

def get_adj(words:str) -> bool:
  print(f'> Прилагательные:')
  adj = show_matches(ADJ, text=words)
  print(f'  > количество: [{len(adj)}]')
  print(f'  > слова: [{adj}]')

def get_count_words(words:str) -> bool:
  print(f'> Слова:')
  words_list = words.strip().split(' ')
  words_list_count = len(words_list)
  if words_list_count <= 5:
      words_list_str = ', '.join(words_list)
  else:
      words_list_str = f'{", ".join(list(words_list)[:5])} и др.'
  words_list_str
  print(f'  > количество: [{len(words_list)}]')
  print(f'  > слова: [{words_list_str}]')

def get_punctuation_marks(words:str) -> bool:
  print(f'> Знаки препинания:')
  marks = '''!()-[]{};?@#$%:'"\,./^;*_'''
  marks_set = set()

  for s in words:
      if s in marks:
          # print(s, True)
          marks_set.add(f'[{s}]')
      else:
          pass
          # print(s, False)
  
  marks_count = len(marks_set)
  if marks_count <= 5:
      marks_str = ', '.join(marks_set)
  else:
      marks_str = f'{", ".join(list(marks_set)[:5])} и др.'
  
  print(f'  > количество: [{marks_count}]')
  print(f'  > знаки: [{marks_str}]')


def create_data_analysis(product_num:int=1, catalog:list=[]) -> bool:

  def title_analysis(title:str) -> bool:
    print(f'\n### Заголовок [{title}]')
    get_count_words(words=title)
    get_punctuation_marks(words=title)
    get_adj(words=title)
    get_colors(words=title)

  def description_analysis(description:str) -> bool:
    print(f'\n### Описание [{description}]')
    get_count_words(words=description)
    get_punctuation_marks(words=description)
    get_adj(words=description)
    get_colors(words=description)
 
  print(f'\n##### АНАЛИЗ ДАННЫХ #####')

  product_id = product_num-1
  name = catalog[product_id]['name']
  description = catalog[product_id]['description']
  subjectName = catalog[product_id]['subjectName']
  chars = catalog[product_id]['chars']
  colors = catalog[product_id]['colors']

  title_analysis(title=name)
  description_analysis(description=description)

print_source_data(product_num)
create_data_analysis(product_num, catalog)
            

##### КАРТОЧКА ТОВАРА #####
> Название товара: [Гигиенические тампоны Compak Super Plus 16шт.]
> Описание товара:
[Чистота останется с тобой, как привязанная! У тампонов Tampax есть гладкий аппликатор, который помогает гигиенично ввести тампон, не касаясь его руками. Одновременно аппликатор позволяет правильно расположить тампон там, где ты практически не будешь его ощущать. Tampax Compak - самый маленький тампон с аппликатором, который так легко спрятать в руке.]
> Название предмета товара: [Тампоны гигиенические]
> Характеристики товара:
  > 1) "Количество предметов в упаковке": [16 шт.]
  > 2) "Транспортировка": [не опасный груз]
  > 3) "Вес товара с упаковкой (г)": [89 г]
  > 4) "Возрастные ограничения": [18+]
  > 5) "Высота упаковки": [6.9 см]
  > 6) "Глубина упаковки": [6.9 см]
  > 7) "Упаковка": [картонная коробка]
  > 8) "Форма упаковки": [без давления]
  > 9) "Ширина упаковки": [9 см]
  > 10) "Страна производства": [Венгрия]
  > 11) "Комплектация": [тампоны]
> Цвета: [темно-си